definition of conctact model 

In [120]:
class ContactDetail:
    def __init__(self, tag: str, detail: str):
        self.tag = tag
        self.detail = detail

class Contact:
    def __init__(self, id: int, family_name: str = "", first_name: str = ""):
        self.id = id
        self.family_name = family_name
        self.first_name = first_name
        self.details = []

data structures needed for contact management

In [121]:
from typing import Iterable

class binary_tree_node:
    def __init__(self, contact: Contact):
        self.contact = contact
        self.left = None
        self.right = None

    def assign(self, node):
        self.contact = node.contact
        self.right = node.right
        self.left = node.left

class naive_hasher:
    def hash(self, id: int) -> int:
        return id

class binary_tree:
    def __init__(self, hasher = naive_hasher()):
        self.__head = None
        self.hasher = hasher

    def getById(self, id: int) -> Contact:
        hash = self.hasher.hash(id)

        node = self.__head
        while node is not None:
            probe = self.hasher.hash(node.contact.id)
            if probe == hash:
                return node.contact
            elif hash < probe:
                node = node.left
            else:
                node = node.right
        return None

    def insert(self, contact: Contact):
        node = binary_tree_node(contact)
        if self.__head is None:
            self.__head = node
            return
        
        hash = self.hasher.hash(node.contact.id)
        return self.__insertAt(node, hash, self.__head)
    
    def remove(self, key: int):
        hash = self.hasher.hash(key)

        parent = None
        node = self.__head
        while node is not None:
            probe = self.hasher.hash(node.contact.id)
            if probe == hash:
                # we found our node. 
                # make sure to promote the children and leave the tree in a valid state. 
                # since our logic is: left < node < right, promoting the right side first 
                # will guarantee that the left side is smaller and therefor the tree stays sorted
                if node.right is not None:
                    left = node.left
                    node.assign(node.right) # reference to node.right now gone. GC will cleanup

                    # relocating the left node
                    if left is not None:
                        self.__insertAt(left, self.hasher.hash(left.contact.id), node)
                elif node.left is not None:
                    node.assign(node.left) # similiar, promote the child and forget original reference 
                else:
                    if parent == None:
                        self.__head = None
                    elif parent.left == node:
                        parent.left = None
                    else:
                        parent.right = None
                return True

            parent = node
            if hash < probe:
                node = node.left
            else:
                node = node.right
        return False

    # traverses the tree with left-first strategy: left-leaf > node > right-leaf
    def travers(self, node: binary_tree_node = None) -> Iterable[Contact]:
        if node is None:
            if self.__head is None:
                return
            node = self.__head

        if node.left is not None:
            yield from self.travers(node.left)
        yield node.contact
        if node.right is not None:
            yield from self.travers(node.right)

    def __insertAt(self, node: binary_tree_node, nodeHash, at: binary_tree_node):
        probe = self.hasher.hash(at.contact.id)
        if probe == nodeHash:
            # Contact with same ID already exists
            return False
        
        if nodeHash < probe:
            if at.left is None:
                at.left = node
                return True
            return self.__insertAt(node, nodeHash, at.left)
        else:
            if at.right is None:
                at.right = node
                return True
            return self.__insertAt(node, nodeHash, at.right)

In [122]:
# test for insertion
tree = binary_tree()
tree.insert(Contact(20))  #           20
tree.insert(Contact(10))  #      10        25
tree.insert(Contact(5))   #   5     15         35
tree.insert(Contact(15)) 
tree.insert(Contact(25))
tree.insert(Contact(35))
assert [x.id for x in tree.travers()] == [5, 10, 15, 20, 25, 35]

tree.insert(Contact(26))
assert [x.id for x in tree.travers()] == [5, 10, 15, 20, 25, 26, 35]

assert tree.insert(Contact(26)) == False # already exists

In [123]:
# test for deletion (same tree as insertion test) 
tree = binary_tree()
tree.insert(Contact(20))  #           20
tree.insert(Contact(10))  #      10        25
tree.insert(Contact(5))   #   5     15         35
tree.insert(Contact(15)) 
tree.insert(Contact(25))
tree.insert(Contact(35))

tree.remove(20)
#           25
#      10        35
#   5     15         
assert [x.id for x in tree.travers()] == [5, 10, 15, 25, 35]

tree.remove(10)
tree.remove(5)
#           25
#      15        35
assert [x.id for x in tree.travers()] == [15, 25, 35]

# test for deletion 2 (left-node relocation)
tree = binary_tree()
tree.insert(Contact(20))  #    20
tree.insert(Contact(25))  #       25
tree.insert(Contact(37))  #           37
tree.insert(Contact(40))  #       34      40
tree.insert(Contact(34))  #     33  35  38 
tree.insert(Contact(33))  
tree.insert(Contact(38))  
tree.insert(Contact(35))
assert [x.id for x in tree.travers()] == [20, 25, 33, 34, 35, 37, 38, 40]

tree.remove(37)
#    20
#       25
#          40
#       38
#    34
#  33  35
assert [x.id for x in tree.travers()] == [20, 25, 33, 34, 35, 38, 40]

# test for deletion (cleanup head)
tree = binary_tree()
tree.insert(Contact(35))
tree.remove(35)
assert [x.id for x in tree.travers()] == []

In [124]:
class dictionary:
    def __init__(self, hasher = naive_hasher()):
        self.numBuckets = 64
        self.buckets = [binary_tree() for _ in range(self.numBuckets)]
        self.hasher = hasher

    def insert(self, c: Contact):
        index = self.hasher.hash(c.id) % self.numBuckets
        bucket = self.buckets[index]
        bucket.insert(c)

    def remove(self, key: int):
        index = self.hasher.hash(key) % self.numBuckets
        bucket = self.buckets[index]
        return bucket.remove(key)

    def get(self, id: int) -> Contact:
        index = self.hasher.hash(id) % self.numBuckets
        bucket = self.buckets[index]
        return bucket.getById(id)

    def values(self) -> Iterable[Contact]:
        for bucket in self.buckets:
            yield from bucket.travers()

In [125]:
# test dictionary insert/remove/get

d = dictionary()
d.insert(Contact(15, "foo", "bar"))
d.insert(Contact(5, "foo2", "bar"))
d.insert(Contact(2, "foo3", "bar"))
d.insert(Contact(32, "foo4", "bar"))

c = d.get(15)
assert c.family_name == "foo" and c.first_name == "bar"

c = d.get(5)
assert c.family_name == "foo2"

c = d.get(2)
assert c.family_name == "foo3"

c = d.get(32)
assert c.family_name == "foo4"

# lookup entry after remove
d.remove(32)
c = d.get(32)
assert c is None

definition of model: contact, contact details, and contact book

In [126]:
def stringContains(text: str, subText: str) -> bool:
    for i in range(len(text)):
        found = True 
        for j in range(len(subText)):
            if subText[j] != text[i+j]:
                found = False
                break
        if found:
            return True
    return False

class ContactBook:
    def __init__(self, initContacts = None):
        self.__id = 1000
        self.__contacts = dictionary()

        for contact in initContacts:
            if contact.id > self.__id:
                self.__id = contact.id
            self.__contacts.insert(contact)

    def add(self, fam_name: str, first_name: str) -> int:
        id = self.__id
        self.__id = self.__id + 1
        self.__contacts.insert(Contact(id, fam_name, first_name))
        return id

    def delete(self, id: int):
        return self.__contacts.remove(id)

    def contacts(self) -> Iterable[Contact]:
        return self.__contacts.values()
    
    def get(self, id):
        return self.__contacts.get(id)

    def search(self, name: str) -> Iterable[Contact]:
        for c in self.contacts():
            match = stringContains(c.first_name, name) or stringContains(c.family_name, name)
            if match:
                yield c

In [127]:
# generator for fake data, generated with ChatGPT
# prompt used:
#
# with following python classes in mind, generate 50 fake persons:
# class ContactDetail:
#     def __init__(self, tag: str, detail: str):
#         self.tag = tag
#         self.detail = detail

# class Contact:
#     def __init__(self, id: int, family_name: str = "", first_name: str = ""):
#         self.id = id
#         self.family_name = family_name
#         self.first_name = first_name
#         self.details = []

import random

# Define sample data for generating fake persons
family_names = ["Smith", "Johnson", "Williams", "Jones", "Brown", "Davis", "Miller", "Wilson", "Taylor", "Anderson"]
first_names = ["James", "Mary", "Robert", "Patricia", "John", "Jennifer", "Michael", "Linda", "David", "Elizabeth"]
tags = ["email", "phone", "address"]
domains = ["example.com", "mail.com", "test.org"]
phone_prefixes = ["+1-202", "+1-303", "+1-404", "+1-505", "+1-606"]
addresses = [
    "123 Main St, Springfield",
    "456 Elm St, Metropolis",
    "789 Maple Ave, Gotham",
    "321 Oak Blvd, Star City",
    "654 Pine Dr, Central City"
]

# Generate random contact details
def generate_contact_details():
    details = []
    num_details = random.randint(1, 3)
    for _ in range(num_details):
        tag = random.choice(tags)
        if tag == "email":
            detail = f"{random.choice(first_names).lower()}.{random.choice(family_names).lower()}@{random.choice(domains)}"
        elif tag == "phone":
            detail = f"{random.choice(phone_prefixes)}-{random.randint(100, 999)}-{random.randint(1000, 9999)}"
        elif tag == "address":
            detail = random.choice(addresses)
        details.append(ContactDetail(tag, detail))
    return details

# Generate fake persons
contacts = []
for i in range(50):
    family_name = random.choice(family_names)
    first_name = random.choice(first_names)
    contact = Contact(id=i + 1, family_name=family_name, first_name=first_name)
    contact.details = generate_contact_details()
    contacts.append(contact)

In [128]:
# user interface


def print_menu():
    print("Menu:")
    print(" q - Quit the application")
    print(" p - Print contacts")
    print(" a - Add a new contact")
    print(" d - Deletes an entry based on the id")
    print(" s - Shows details of given entry id")
    print(" f - Find contacts for a name or partial name")


def validate_name_input(name: str) -> bool:
    if len(name) == 0:
        print("Name must not be empty!")
        return False
    if not name.isalpha():
        print("Name must only contain letters!")
        return False
    return True


def add_contact(book: ContactBook):
    fam_name = input("Enter Family Name: ")
    if not validate_name_input(fam_name):
        return

    first_name = input("Enter First Name: ")
    if not validate_name_input(first_name):
        return

    id = book.add(fam_name, first_name)
    print("New contact added with id {}".format(id))


def print_contacts(contacts: Iterable[Contact]):
    headers = ["Id", "Family Name", "First Name"]
    data = [(str(x.id), x.family_name, x.first_name) for x in contacts]

    # determine formatted text length to pad the table columns correctly
    max_col_len_hdr = [len(x) for x in headers]
    max_col_len = max_col_len_hdr
    for row in data:
        row_col_len = [len(x) for x in row]
        max_col_len = [max(*len) for len in zip(max_col_len, row_col_len)]

    # 'ljust()' to pad the strings to align with column width
    print(" | ".join([x.ljust(max_col_len[i], " ") for i, x in enumerate(headers)]))
    print("-".ljust(sum(max_col_len) + (len(" | ") * (len(headers) - 1)), "-"))
    for contact in data:
        print(" | ".join([v.ljust(max_col_len[i], " ") for i, v in enumerate(contact)]))

def delete_entry(book: ContactBook):
    id = input("Enter ID of entry to be removed")
    if not id.isnumeric():
        print("id must be a number")
        return

    if book.delete(int(id)):
        print("entry has been removed from contact book")
    else:
        print("no entry with given id found")

def show_details(book: ContactBook):
    id = input("Enter ID of entry")
    if not id.isnumeric():
        print("id must be a number")
        return
    
    c = book.get(int(id))
    if c is None:
        print("entry not found")
        return

    print(c.first_name, c.family_name)
    for x in c.details:
        print(f"{x.tag}: {x.detail}")

def find_contacts(book: ContactBook):
    name = input("Enter name of person")
    if not name.isalpha():
        print("input must be only letters")
        return
    
    print_contacts(book.search(name))

book = ContactBook(contacts)
while True:
    print_menu()

    inp = input("> ")
    exit = inp in ["q", "quit", "exit"]
    if exit:
        break

    print()
    match inp:
        case "p":
            print_contacts(book.contacts())
        case "a":
            add_contact(book)
        case "d":
            delete_entry(book)
        case "s":
            show_details(book)
        case "f":
            find_contacts(book)
    print()

Menu:
 q - Quit the application
 p - Print contacts
 a - Add a new contact
 d - Deletes an entry based on the id
 s - Shows details of given entry id
 f - Find contacts for a name or partial name

Id | Family Name | First Name
-----------------------------
1  | Smith       | Elizabeth 
2  | Anderson    | John      
3  | Taylor      | Jennifer  
4  | Williams    | Patricia  
5  | Wilson      | Jennifer  
6  | Wilson      | John      
7  | Johnson     | Jennifer  
8  | Johnson     | David     
9  | Johnson     | Patricia  
10 | Taylor      | Patricia  
11 | Johnson     | Robert    
12 | Johnson     | Elizabeth 
13 | Anderson    | Mary      
14 | Williams    | Mary      
15 | Miller      | Patricia  
16 | Jones       | Michael   
17 | Williams    | Jennifer  
18 | Anderson    | Patricia  
19 | Williams    | Jennifer  
20 | Davis       | Linda     
21 | Smith       | Mary      
22 | Davis       | John      
23 | Jones       | Jennifer  
24 | Brown       | John      
25 | Johnson     | Robe